In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

2024-09-15 16:07:29.343228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 16:07:29.351431: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 16:07:29.353883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 16:07:29.360350: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 16:07:29.812630: W tensorflow/compiler/tf2

In [2]:
# 1. Load the MNIST dataset
# MNIST consists of 60,000 training images and 10,000 test images of handwritten digits (0-9)
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
# 2. Preprocess the data
# Reshape the images to have a single channel and normalize pixel values
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [4]:
# Convert labels to categorical (one-hot encoding)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [5]:
# 3. Data Augmentation: Create an ImageDataGenerator to apply transformations
datagen = ImageDataGenerator(
    rotation_range=40,         # Rotate images randomly by up to 40 degrees
    width_shift_range=0.2,     # Shift images horizontally by up to 20%
    height_shift_range=0.2,    # Shift images vertically by up to 20%
    shear_range=0.2,           # Shear transformation by up to 20%
    zoom_range=0.2,            # Randomly zoom in/out by up to 20%
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill empty pixels with the nearest value
)

In [6]:
# 4. Build the CNN model
model = models.Sequential([
    # First convolutional layer: 32 filters, 3x3 size, ReLU activation
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    # Max pooling layer: Reduce the spatial dimensions by taking the max in each 2x2 block
    layers.MaxPooling2D((2, 2)),
    
    # Second convolutional layer: 64 filters, 3x3 size, ReLU activation
    layers.Conv2D(64, (3, 3), activation='relu'),
    # Another max pooling layer
    layers.MaxPooling2D((2, 2)),

    # Third convolutional layer: 64 filters, 3x3 size, ReLU activation
    layers.Conv2D(64, (3, 3), activation='relu'),

    # Flatten the 3D feature maps into a 1D vector
    layers.Flatten(),
    
    # Fully connected layer with 64 units, ReLU activation
    layers.Dense(64, activation='relu'),

    # Output layer with 10 units (one for each digit), softmax activation for probability prediction
    layers.Dense(10, activation='softmax')
])

/home/woong/myenv/lib/python3.9/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1726409256.316494   43031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726409256.331378   43031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726409256.331421   43031 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1726409256.334314   43031 cuda_executor.cc:

In [7]:
# 5. Compile the model
# Using Adam optimizer, sparse categorical crossentropy loss, and accuracy as the metric
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6. Train the model using data augmentation
# Fit the model using the augmented data from the ImageDataGenerator
# train_datagen.flow() generates augmented images in batches during training
model.fit(datagen.flow(train_images, train_labels, batch_size=64), 
          epochs=5, 
          validation_data=(test_images, test_labels))

# 7. Evaluate the model
# Evaluate the model on the test set and print the accuracy
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy with data augmentation: {test_acc}")

print(test_labels)

Epoch 1/5


/home/woong/myenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726409261.344197   44313 service.cc:146] XLA service 0x7f9e10003e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726409261.344232   44313 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2024-09-15 16:07:41.359985: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-15 16:07:41.439658: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 25/938 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - accuracy: 0.1326 - loss: 2.2962

I0000 00:00:1726409262.605037   44313 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.4555 - loss: 1.5507 - val_accuracy: 0.9003 - val_loss: 0.3195
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8017 - loss: 0.6105 - val_accuracy: 0.9390 - val_loss: 0.2039
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8590 - loss: 0.4404 - val_accuracy: 0.9529 - val_loss: 0.1642
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8796 - loss: 0.3765 - val_accuracy: 0.9576 - val_loss: 0.1318
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.8937 - loss: 0.3327 - val_accuracy: 0.9600 - val_loss: 0.1271
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9530 - loss: 0.1425
Test accuracy with data augmentation: 0.9599999785423279
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Force TensorFlow to use CPU
with tf.device('/CPU:0'):

    # 1. Load the MNIST dataset
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

    # 2. Preprocess the data: Reshape and normalize the images
    train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
    test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255

    # Convert labels to categorical (one-hot encoding)
    train_labels = to_categorical(train_labels)
    test_labels = to_categorical(test_labels)

    # 3. Data Augmentation setup
    datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # 4. Build the CNN model
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax')  # Output layer with 10 units (0-9 digits)
    ])

    # 5. Compile the model
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    # 6. Train the model using data augmentation
    model.fit(datagen.flow(train_images, train_labels, batch_size=64), 
              epochs=5, 
              validation_data=(test_images, test_labels))

    # 7. Evaluate the model on test data
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print(f"Test accuracy with data augmentation on CPU: {test_acc}")


2024-09-15 16:01:08.762152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 16:01:08.808284: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 16:01:08.823354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 16:01:08.901649: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 16:01:09.700263: W tensorflow/compiler/tf2

Epoch 1/5


/home/woong/myenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726408871.764270   39741 service.cc:146] XLA service 0x7fb2b4003b10 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726408871.764319   39741 service.cc:154]   StreamExecutor device (0): Host, Default Version
2024-09-15 16:01:11.791959: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


  7/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.1206 - loss: 2.2957

I0000 00:00:1726408872.190777   39741 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


938/938 ━━━━━━━━━━━━━━━━━━━━ 22s 22ms/step - accuracy: 0.4571 - loss: 1.5229 - val_accuracy: 0.9111 - val_loss: 0.2842
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.7914 - loss: 0.6411 - val_accuracy: 0.9395 - val_loss: 0.1909
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 22ms/step - accuracy: 0.8549 - loss: 0.4530 - val_accuracy: 0.9471 - val_loss: 0.1670
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.8790 - loss: 0.3842 - val_accuracy: 0.9557 - val_loss: 0.1386
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 20s 21ms/step - accuracy: 0.8927 - loss: 0.3270 - val_accuracy: 0.9548 - val_loss: 0.1502
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9479 - loss: 0.1740
Test accuracy with data augmentation on CPU: 0.954800009727478
